# WORD EMBEDDING

## 参考资料

[什么是 word embedding?](https://www.zhihu.com/question/32275069/answer/61059440)

[WORD EMBEDDINGS: ENCODING LEXICAL SEMANTICS](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#sphx-glr-beginner-nlp-word-embeddings-tutorial-py)

[YJango的Word Embedding--介绍](https://zhuanlan.zhihu.com/p/27830489)

## 概念解析

## 代码解析

与我们制作独热向量（one-hot vectors）时候为每个单词定义一个唯一索引类似，在使用词嵌入的时候我们也需要为每个单词定义索引，方便查找。

词嵌入被储存为了一个 $ |V|×D $ 的矩阵，其中 $ D $ 是嵌入的维度，所以某单词的索引为 $ i $，那么它就被嵌入在矩阵的第 $ i $ 行中。在代码中，从单词到索引的映射是一个名叫 word_to_ix 的字典。

在 PyTorch 中，torch.nn.Embedding 可以完成词嵌入功能，包含两个参数：词表大小和嵌入维度。

使用 torch.LongTensor 进行索引（这是一个 64-bit integer，带符号数）。

In [1]:
# Author: Robert Guthrie
# 补充：Dr_David_S

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)  # 设定随机种子，返回一个torch.Generator对象

现在我们设置一个字典，用最简单的 hello world 做例子：

In [2]:
word_to_ix = {"hello": 0, "world": 1}
word_to_ix

{'hello': 0, 'world': 1}

使用 torch.nn.Embedding 方法准备进行词嵌入，参数的意义是：有2个词语，5个维度。

In [3]:
embeds = nn.Embedding(num_embeddings=2, embedding_dim=5)  # 2 words in vocab, 5 dimensional embeddings
embeds

Embedding(2, 5)

现在将 hello 对应的值 0 转换为一个 longtensor 类型,两种方法都行。

In [4]:
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
lookup_tensor

tensor([0])

In [5]:
lookup_tensor_test = torch.LongTensor([word_to_ix["hello"]])
lookup_tensor_test

tensor([0])

现在使用之前定义的 embeds 模型对 hello 这个次进行词嵌入（或者说转换为词向量）：

In [6]:
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


原来的代码到此就结束了，但是这里我们希望再试试对 world 的词嵌入：

In [7]:
lookup_tensor2 = torch.tensor([word_to_ix["world"]], dtype=torch.long)
lookup_tensor2

tensor([1])

In [8]:
world_embed = embeds(lookup_tensor2)
print(world_embed)

tensor([[-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]],
       grad_fn=<EmbeddingBackward>)


从上述结果来看，似乎对于 hello 的 embedding 没有涉及到 world？

事实上，embeds只是一个随机生成的矩阵，目前其中的向量并不能代表任何意义，如下：

In [9]:
embeds.weight

Parameter containing:
tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519],
        [-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]], requires_grad=True)

>nn.Embedding.weight初始化分布符合标准正态分布 $ N(0,1) $，即均值 $\mu=0$，方差 $\sigma=1$ 的正态分布。
>
>因为这里的初始化参数只设定了两个单词，所以其中：
>
>某一行代表单词 'hello'
>
>另一行代表单词 'world'

## 一个 N-Gram 语言模型

之前我们的 embeds 矩阵的初始值是随机生成的，现在我们将要计算一些训练样例的损失函数，然后使用反向传播去更新参数。

### 文本处理

首先设定参数：

In [10]:
CONTEXT_SIZE = 2  # 指上下文的size
EMBEDDING_DIM = 10

其中，上下文是指目标单词的前 $ n $ 个单词，或者后 $ n $ 个单词，或者前后都有总共 $ n $ 个单词。

在这里，CONTEXT_SIZE 指的是上文的两个单词，意味着如果以后遇到上下文的情况，需要乘以二。

EMBEDDING_DIM 指的是会将其 embedding 为十维向量。

在本例中，我们使用的是莎士比亚的十四行诗，使用空格分词：

In [11]:
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

>SONNET 2：
>
>当四十个冬天围攻你的朱颜，
>
>在你美的园地挖下深的战壕，
>
>你青春的华服，那么被人艳羡，
>
>将成褴褛的败絮，谁也不要瞧：
>
>那时人若问起你的美在何处，
>
>哪里是你那少壮年华的宝藏，
>
>你说，"在我这双深陷的眼眶里，
>
>是贪婪的羞耻，和无益的颂扬。"
>
>你的美的用途会更值得赞美，
>
>如果你能够说，"我这宁馨小童
>
>将总结我的账，宽恕我的老迈，"
>
>证实他的美在继承你的血统！
>
>这将使你在衰老的暮年更生，
>
>并使你垂冷的血液感到重温。
>

In [12]:
test_sentence

['When',
 'forty',
 'winters',
 'shall',
 'besiege',
 'thy',
 'brow,',
 'And',
 'dig',
 'deep',
 'trenches',
 'in',
 'thy',
 "beauty's",
 'field,',
 'Thy',
 "youth's",
 'proud',
 'livery',
 'so',
 'gazed',
 'on',
 'now,',
 'Will',
 'be',
 'a',
 "totter'd",
 'weed',
 'of',
 'small',
 'worth',
 'held:',
 'Then',
 'being',
 'asked,',
 'where',
 'all',
 'thy',
 'beauty',
 'lies,',
 'Where',
 'all',
 'the',
 'treasure',
 'of',
 'thy',
 'lusty',
 'days;',
 'To',
 'say,',
 'within',
 'thine',
 'own',
 'deep',
 'sunken',
 'eyes,',
 'Were',
 'an',
 'all-eating',
 'shame,',
 'and',
 'thriftless',
 'praise.',
 'How',
 'much',
 'more',
 'praise',
 "deserv'd",
 'thy',
 "beauty's",
 'use,',
 'If',
 'thou',
 'couldst',
 'answer',
 "'This",
 'fair',
 'child',
 'of',
 'mine',
 'Shall',
 'sum',
 'my',
 'count,',
 'and',
 'make',
 'my',
 'old',
 "excuse,'",
 'Proving',
 'his',
 'beauty',
 'by',
 'succession',
 'thine!',
 'This',
 'were',
 'to',
 'be',
 'new',
 'made',
 'when',
 'thou',
 'art',
 'old,',
 

按道理，我们在训练之前输入的数据应当是有标记的，我们暂时忽略这个事情（后面会说明）。

然后我们要建立一个元组（tuple），元组的格式为：

$$ ([word_{i-2}, word_{i-1}], word_{i}) $$

其中 $word_{i}$ 就是目标词 $ target word $ 

根据以上方法遍历整首诗，然后生成多个元组，放入列表 trigrams 中。

In [13]:
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


set() 函数创建一个无序不重复元素集，可以剔除掉重复的元素，同时可能可以在部分模型如 RNN 中打乱文档上下文，忽略掉 RNN 权重偏向后输入的词语的缺点。

In [14]:
vocab = set(test_sentence)  # 去重
word_to_ix = {word: i for i, word in enumerate(vocab)}

### 搭建网络

接下来我们构建一个网络，名叫 NGramLanguageModeler：

In [15]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)  # 这步实现了embedding
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

从上述网络来看，init 中存在三个层，分别是：

- 词嵌入层（embedding层）
- 第一个全连接层（linear1）
- 第二个全连接层（linear2）

算上前向传播的过程就是：

- 词嵌入层
- 转换为一维
- 第一个全连接层
- ReLU激活
- 第二个全连接层
- log_softmax输出

### 设置损失函数和优化函数

- 使用NLLLoss()，这个函数其实就是一个不带 log_softmax 的交叉熵损失函数。

- 使用optim.SGD()，最传统的随机梯度下降，没什么好说的。

需要注意的是，model 的输入参数有三个，第一个就是去重后的上下文三词元组数量，第二三个则是我们设定好的 CONTEXT_SIZE 和 EMBEDDING_DIM。

In [16]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

### 训练模型

注意：
```Python
for context, target in trigrams
```
这句话指的是我们的训练样本是上下文（其实只有上文），标签是当前词。这也是为什么之前没有标记的原因。

In [17]:
trigrams

[(['When', 'forty'], 'winters'),
 (['forty', 'winters'], 'shall'),
 (['winters', 'shall'], 'besiege'),
 (['shall', 'besiege'], 'thy'),
 (['besiege', 'thy'], 'brow,'),
 (['thy', 'brow,'], 'And'),
 (['brow,', 'And'], 'dig'),
 (['And', 'dig'], 'deep'),
 (['dig', 'deep'], 'trenches'),
 (['deep', 'trenches'], 'in'),
 (['trenches', 'in'], 'thy'),
 (['in', 'thy'], "beauty's"),
 (['thy', "beauty's"], 'field,'),
 (["beauty's", 'field,'], 'Thy'),
 (['field,', 'Thy'], "youth's"),
 (['Thy', "youth's"], 'proud'),
 (["youth's", 'proud'], 'livery'),
 (['proud', 'livery'], 'so'),
 (['livery', 'so'], 'gazed'),
 (['so', 'gazed'], 'on'),
 (['gazed', 'on'], 'now,'),
 (['on', 'now,'], 'Will'),
 (['now,', 'Will'], 'be'),
 (['Will', 'be'], 'a'),
 (['be', 'a'], "totter'd"),
 (['a', "totter'd"], 'weed'),
 (["totter'd", 'weed'], 'of'),
 (['weed', 'of'], 'small'),
 (['of', 'small'], 'worth'),
 (['small', 'worth'], 'held:'),
 (['worth', 'held:'], 'Then'),
 (['held:', 'Then'], 'being'),
 (['Then', 'being'], 'asked

In [18]:
for epoch in range(100):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        # 这一步把上下文单词转换为字典中对应单词的值，作为tensor
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[521.878231048584, 519.3365745544434, 516.814404964447, 514.3094692230225, 511.820321559906, 509.3474576473236, 506.8883717060089, 504.44230675697327, 502.0088152885437, 499.5859134197235, 497.1743552684784, 494.77218222618103, 492.3794560432434, 489.99517035484314, 487.6185758113861, 485.2484574317932, 482.88393354415894, 480.52435755729675, 478.17004323005676, 475.8179910182953, 473.46933341026306, 471.12329840660095, 468.77851271629333, 466.4353778362274, 464.09136986732483, 461.7471446990967, 459.4029574394226, 457.0576317310333, 454.7120842933655, 452.36570167541504, 450.0161933898926, 447.6633388996124, 445.30811834335327, 442.9496729373932, 440.58748269081116, 438.2198576927185, 435.84934091567993, 433.47243475914, 431.08960461616516, 428.7008910179138, 426.3039280176163, 423.9011753797531, 421.4890441894531, 419.06925213336945, 416.6411648988724, 414.2071763277054, 411.7645524740219, 409.31272637844086, 406.85250759124756, 404.38154780864716, 401.9022305011749, 399.413685560226

现在我们得到了一个损失逐渐降低的模型，可以看出这个模型的结果是收敛的，如果继续训练下去，可能会更准确，但是到此为止吧。

## 练习：计算词嵌入：CBOW模型

连续词袋模型（Continuous Bag-of-Words model ），又被称为CBOW模型，经常用于自然语言处理深度学习。

CBOW模型使用一段文本的中间词作为目标词，即给出目标词的上下文单词作为训练数据。CBOW去除了上下文各词语的词序信息，使用的是上下文各词的词向量的平均值。

CBOW对于一段长度为 $ n $ 的训练样本：$ w_{i-(n-1)}, ...,w_i $ ，其输入为: $$ x = \frac {1}{n - 1}\sum_{w_j \in c}e(w_j) $$

其中 $ w $ 是目标词，即 $ w_i $， $ c $ 是上下文，上式就是将输入求平均。

故CBOW模型根据上下文的表示，直接对目标词进行预测： $$ P(w|c) = \frac{\exp(e'(w)^Tx)}{\sum_{w'\in V}\exp \lgroup e'(w')^Tx\rgroup} $$ 

CBOW的目标是最大化 $$ \sum_{(w,c)\in D}\log P(w|c) $$

其中 $D$ 指的是整段语料（或者说整篇文章）。

要在PyTorch中实现这个模型，需要注意的是：

- 考虑一下需要定义哪些参数
- 确保你清楚每一步需要哪些形状的矩阵，使用 .view() 函数来 reshape 矩阵

### 1.分割语料

代码示例如下，第一步，先按空格分隔语料中的单词，其中 2 表示上文两个单词加下文两个单词。

In [19]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

使用 set() 函数将分词唯一化并打乱顺序，然后建立一个名叫 word_to_ix 的字典，字典的 key 是单词，value 是单词的索引。

接下来使用data列表来储存文本和目标了，把第 $i$ 个单词作为目标，把第$[i-2, i-1, i+1, i+2]$ 个单词作为上下文（特征），注意这里使用的是有序的语料。

In [20]:
# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    # context保存目标单词上下文，分别2个单词
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    # target保存目标单词
    target = raw_text[i]
    # 存入data列表中
    data.append((context, target))
    
print(data[:5])
print(len(data))
print(vocab_size)

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]
58
49


可以看出我们的上下文目标词组有58个，但是非重复单词有49个。

### 2.建立CBOW类

建立 CBOW 类，由于本节有习题性质，CBOW 类并没有内容，需要自己填空。

In [21]:
class CBOW(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    """从字典中抽取word对应的value
    
    输入：
    context:上下文列表（四个单词）
    word_to_ix:单词的字典
    
    输出：context中的单词对应的longtensor
    """
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


first_context = make_context_vector(data[0][0], word_to_ix)  # example
print(first_context)

tensor([33, 17, 36, 37])


这里给了一个例子，就是

```Python
make_context_vector(data[0][0], word_to_ix)  # example
```
简单分析一下：

```Python
data[0][0]
```
指的就是前面 data 列表中第一个tuple中的第一个元素，即:

In [22]:
print(data[0][0])

['We', 'are', 'to', 'study']


上述四个单词在字典 word_to_ix 中对应的值如下：

In [23]:
print(word_to_ix['We'])
print(word_to_ix['are'])
print(word_to_ix['to'])
print(word_to_ix['study'])

33
17
36
37


故其组成的 longtensor 自然就是 tensor([20, 27, 26, 21]) 啦~

### 简单embedding演示

现在，使用 torch.nn.Embedding 方法建立一个embedding模型，目标是去重后的49个词，每个词20维。

In [24]:
embeds = nn.Embedding(num_embeddings=49, embedding_dim=20)  # 58 words in vocab, 20 dimensional embeddings
embeds

Embedding(49, 20)

我们尝试使用上面的 embeds 模型将 first_context 向量化：

In [25]:
first_context_embed = embeds(first_context)
print(first_context_embed)

tensor([[-0.2718, -1.6699,  2.0873, -0.5097, -0.4212, -0.9803,  0.7608, -0.8248,
          1.0795, -0.7436, -1.1971, -0.8597,  0.6141, -0.0859, -0.2654,  1.8039,
         -0.0279,  1.2599, -0.5165,  0.3539],
        [ 1.8870, -0.3962,  1.3817, -1.1175,  0.1472, -1.4784, -0.4961,  0.8160,
         -1.4932, -1.0810, -0.8416,  1.2265, -0.4613, -1.1796, -2.0502, -1.4492,
         -0.3409, -2.8001, -0.3261,  1.0929],
        [-1.0197, -0.2241,  1.2898, -0.6360, -1.6775, -0.3627, -0.1158, -0.7729,
          0.6541, -1.0522,  0.6729,  2.1099,  0.1009,  0.8126,  0.8929,  0.2729,
         -0.0568, -0.7972, -0.3836,  2.1871],
        [ 1.3518,  0.6369, -1.5268,  0.4858, -0.6772,  1.0290,  1.2745,  0.6527,
         -0.3394,  0.4518, -2.0870,  0.8774, -1.5316,  1.6614, -0.9597, -1.0855,
          1.3273, -0.6726,  0.6029, -0.5115]], grad_fn=<EmbeddingBackward>)


可以看出已经成功的完成了我们向量化的第一步实验，接下来的任务就是把所有的上下文和目标单词向量化，然后建立一个 CBOW 模型对这些向量进行训练。

顺便看看 embeds 的初始化，应当是随机初始化了一个符合标准正态分布 $ N(0,1) $的 49 行，20 列的矩阵。

In [26]:
embeds.weight

Parameter containing:
tensor([[-6.4612e-01,  3.2699e-01, -1.2190e+00, -5.4798e-01, -1.7273e+00,
         -7.3260e-01,  6.2941e-01, -2.3110e-01,  4.3045e-01,  1.7503e+00,
         -2.9139e-01, -4.2367e-01,  5.4413e-01,  1.6597e+00, -5.6447e-01,
         -7.9011e-01,  4.2483e-01, -2.5986e+00, -9.2568e-01, -8.6511e-01],
        [-1.6726e-01,  1.5749e+00, -1.1857e+00,  1.2867e+00, -5.1799e-01,
          2.3175e+00, -1.9279e+00,  1.2128e+00,  7.7894e-01,  3.8469e-02,
         -1.1871e+00,  3.4312e-01,  6.9343e-01,  1.0216e+00, -7.4495e-01,
          4.6306e-02, -1.5447e+00, -1.5220e+00,  9.3887e-01, -5.8114e-01],
        [ 1.9286e+00, -1.0957e+00,  6.8782e-01, -5.4693e-01, -5.5047e-01,
          5.0876e-01,  8.9655e-01,  4.8738e-01, -2.6993e-01,  3.3697e-01,
          3.7492e-01, -3.6387e-01, -5.9859e-02,  8.9039e-01,  1.6794e-01,
         -1.8218e+00, -2.9632e-01,  2.2457e-01,  6.6170e-01,  1.2258e+00],
        [ 1.5298e-01,  3.1138e-01,  4.5681e-01,  6.1812e-01,  5.5626e-01,
          8.4

这里只是说明一下，而后我们将补全之前的 CBOW 类代码，遍历 data 中的每个元素进行embedding。

### 补全CBOW类

事实上，我们对目标词和其上下文的 embedding 是在 CBOW 类中完成的。我们基于之前的示例代码进行修改。

In [27]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        """
        层搭建
        
        输入：
        vocab_size：去重后的单词量，本例为49个
        embedding_dim：需要 embedding 的维度，本例设为20维
        context_size：上下文的大小，注意和前个例子不一样，前面是上文，这里是上下文
        """
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * 2 * embedding_dim, 128)  # 这里乘以2应该是上下文的原因
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        """
        前向传播
        
        输入：
        inputs：
        """
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)  # 输出概率
        return log_probs  # 结果

### 设置损失函数

同前个例子

In [28]:
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

### 开始训练

In [29]:
for epoch in range(100):
    total_loss = 0
    for context, target in data:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        # 这一步把上下文单词转换为字典中对应单词的值，作为tensor
        
        context_idxs = make_context_vector(context=context, word_to_ix=word_to_ix)
        # context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        # 梯度归零
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        # 输出概率
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        # 这里target其实是一个数字标签，通过数字标签在字典中倒推回单词。
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[228.1266691684723, 226.76646089553833, 225.41565012931824, 224.0730266571045, 222.73823070526123, 221.41210842132568, 220.09397792816162, 218.78319811820984, 217.47804927825928, 216.17788124084473, 214.88398122787476, 213.59356570243835, 212.30941009521484, 211.028706073761, 209.75223588943481, 208.47870922088623, 207.20958256721497, 205.94271397590637, 204.6795690059662, 203.41753602027893, 202.15840220451355, 200.90097188949585, 199.6455478668213, 198.3940029144287, 197.14317846298218, 195.89293932914734, 194.6439757347107, 193.394837141037, 192.14903092384338, 190.90403127670288, 189.6587804555893, 188.41311919689178, 187.16708159446716, 185.9181423187256, 184.67125380039215, 183.4199755191803, 182.16915893554688, 180.91770470142365, 179.66470193862915, 178.41098296642303, 177.15614020824432, 175.9006907939911, 174.6447789669037, 173.39031636714935, 172.13246071338654, 170.87402880191803, 169.61321783065796, 168.35220444202423, 167.09121692180634, 165.82751560211182, 164.5627216100

In [30]:
len(losses)

100

In [31]:
torch.cuda.is_available()

True